<a href="https://colab.research.google.com/github/saribasmetehan/Create-custom-ner-model-with-Spacy/blob/main/ner7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import ibraries**

In [2]:
import random
import spacy
from spacy.training import Example
import json
import requests
import random
import spacy
from spacy.training import Example

**GPU checging**

In [3]:
spacy.prefer_gpu()

True

**Upload dataset**

In [5]:
url = "https://raw.githubusercontent.com/turkish-nlp-suite/Atis_Turkish/main/json/atis_train.json"

response = requests.get(url)

with open("atis_train.json", "wb") as f:
    f.write(response.content)

with open("atis_train.json") as f:
    data = json.load(f)

TRAIN_DATA = []
for item in data:
    text = item["text"]
    entities = [(entity["start"], entity["end"], entity["entity"]) for entity in item["entities"]]
    TRAIN_DATA.append((text, {"entities": entities}))

In [6]:
TRAIN_DATA

[("Pittsburgh'tan Atlanta'ya 25 Nisan'da gidiş 6 Mayıs'ta dönüşü olan bir gidiş dönüş uçuşunun maliyeti nedir",
  {'entities': [(0, 14, 'fromloc.city_name'),
    (15, 25, 'toloc.city_name'),
    (26, 28, 'depart_date.day_number'),
    (29, 37, 'depart_date.month_name'),
    (44, 45, 'return_date.day_number'),
    (46, 54, 'return_date.month_name')]}),
 ("Şimdi Fort Worth'dan ayrılan ve en geç gelecek pazartesi akşam 2'ye kadar Denver'e varacak bir uçağa ihtiyacım var",
  {'entities': [(6, 20, 'fromloc.city_name'),
    (39, 46, 'arrive_date.date_relative'),
    (47, 56, 'arrive_date.day_name'),
    (57, 62, 'arrive_time.period_of_day'),
    (63, 67, 'arrive_time.time'),
    (74, 82, 'toloc.city_name')]}),
 ("Önümüzdeki çarşamba gidiş ertesi gün dönüşü olan Kansas City'den Chicago'ya giden bir uçuşa ihtiyacım var",
  {'entities': [(0, 10, 'depart_date.date_relative'),
    (11, 19, 'depart_date.day_name'),
    (26, 36, 'return_date.today_relative'),
    (49, 55, 'fromloc.state_name'),
   

**Creating nlp structure**

In [7]:
nlp = spacy.blank("tr")
ner = nlp.add_pipe("ner")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [8]:
ner

In [9]:
nlp.meta

{'lang': 'tr',
 'name': 'pipeline',
 'version': '0.0.0',
 'spacy_version': '>=3.7.4,<3.8.0',
 'description': '',
 'author': '',
 'email': '',
 'url': '',
 'license': '',
 'spacy_git_version': 'bff8725f4',
 'vectors': {'width': 0,
  'vectors': 0,
  'keys': 0,
  'name': None,
  'mode': 'default'},
 'labels': {'ner': []},
 'pipeline': ['ner'],
 'components': ['ner'],
 'disabled': []}

**Appending labels**

In [10]:
labels = ["abbreviation",
"aircraft",
"airfare",
"airline",
"airport",
"capacity",
"city",
"distance",
"flight",
"flight_no",
"flight_time",
"ground_fare",
"ground_service",
"meal",
"quantity",
"restrictionaircraft_code",
"airline_code",
"airline_name",
"airport_code",
"airport_name",
"arrive_date.date_relative",
"arrive_date.day_name",
"arrive_date.day_number",
"arrive_date.month_name",
"arrive_date.today_relative",
"arrive_end_time.time",
"arrive_start_time.time",
"arrive_time.day_number",
"arrive_time.period_of_day",
"arrive_time.time",
"arrive_time.time_relative",
"city_name",
"class_type",
"connect",
"cost_relative",
"date_relative",
"day_name",
"day_number",
"depart_date.date_relative",
"depart_date.day_name",
"depart_date.day_number",
"depart_date.month_name",
"depart_date.today_relative",
"depart_date.year",
"depart_day.day_number",
"depart_end_time.time",
"depart_start_time.time",
"depart_time.period_of_day",
"depart_time.time",
"depart_time.time_relative",
"fare_amount",
"fare_basis_code",
"flight_mod",
"flight_number",
"flight_stop",
"flight_time",
"fromloc.airport_code",
"fromloc.airport_name",
"fromloc.city_name",
"fromloc.state_code",
"fromloc.state_name",
"meal",
"meal_code",
"meal_description",
"mod",
"month_name",
"or",
"part_date.day_number",
"period_of_day",
"restriction_code",
"return_date.date_relative",
"return_date.day_name",
"return_date.day_number",
"return_date.month_name",
"return_date.today_relative",
"return_end_time.time",
"return_time.period_of_day",
"return_time.time",
"round_trip",
"state_code",
"state_name",
"stoploc.airport_name",
"stoploc.city_name",
"stoploc.ity_name",
"stoploc.state_code",
"today_relative",
"toloc.airport_code",
"toloc.airport_name",
"toloc.city_name",
"toloc.country_name",
"toloc.state_code",
"toloc.state_name",
"transport_type",]

for ent in labels:
 ner.add_label(ent)

print(ner.labels)

('abbreviation', 'aircraft', 'airfare', 'airline', 'airline_code', 'airline_name', 'airport', 'airport_code', 'airport_name', 'arrive_date.date_relative', 'arrive_date.day_name', 'arrive_date.day_number', 'arrive_date.month_name', 'arrive_date.today_relative', 'arrive_end_time.time', 'arrive_start_time.time', 'arrive_time.day_number', 'arrive_time.period_of_day', 'arrive_time.time', 'arrive_time.time_relative', 'capacity', 'city', 'city_name', 'class_type', 'connect', 'cost_relative', 'date_relative', 'day_name', 'day_number', 'depart_date.date_relative', 'depart_date.day_name', 'depart_date.day_number', 'depart_date.month_name', 'depart_date.today_relative', 'depart_date.year', 'depart_day.day_number', 'depart_end_time.time', 'depart_start_time.time', 'depart_time.period_of_day', 'depart_time.time', 'depart_time.time_relative', 'distance', 'fare_amount', 'fare_basis_code', 'flight', 'flight_mod', 'flight_no', 'flight_number', 'flight_stop', 'flight_time', 'fromloc.airport_code', 'from

**Training**

In [ ]:
epochs = 100
other_pipes = [pipe for pipe in nlp.pipe_names if pipe !=
'ner']
with nlp.disable_pipes(*other_pipes):
  optimizer = nlp.begin_training()
  for i in range(epochs):
    random.shuffle(TRAIN_DATA)
    for text, annotation in TRAIN_DATA:
      doc = nlp.make_doc(text)
      example = Example.from_dict(doc, annotation)
      nlp.update([example], sgd=optimizer)

**Saving**

In [11]:
nlp.to_disk("custom_ner")

**Loading and using**

In [15]:
nlp = spacy.load("/content/drive/MyDrive/custom_ner/custom_ner")

text = "Lütfen Malatya'dan İstanbul'a uçmak istiyorum"
doc = nlp(text)
for ent in doc.ents:
    print(ent.text,ent.label_ )


Malatya'dan fromloc.city_name
İstanbul'a toloc.city_name
